In [ ]:
# Análisis e Implementación en SparkQL

* Entrega hasta el 24 de mayo en https://forms.gle/aRiWtjayausRFCTd8

* Los dos conjuntos de datos entregados, CSV separados por coma, Evaluación del Pitch (2021.04.13 ISoftware).csv y Asistencia del Pitch (2021.04.13 ISoftware).csv proceden de dos encuestas realizadas con Google Forms.

* Los archivos proceden de las evaluaciones de los pitch (exposiciones cortas de negocio) que realizan los estudiantes de la asignatura de Ingeniería de Software. Los estudiantes están organizados en equipos. Cada equipo realiza una presentación de máximo 15 minutos, una vez inicia la presentación todos los estudiantes deben registrar su asistencia lo cual queda registrado en 'Asistencia del Pitch (2021.04.13 ISoftware).csv' incluidos los miembros del equipo.

* El archivo `Asistencia del Pitch (2021.04.13 ISoftware).csv` contiene las columnas: `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto,`"Equipo al que perteneces:"` equipo de trabajo al que pertenece el estudiante también es texto y `"Equipo que va a exponer:"` equipo que el estudiante va a tender a su presentación.

* El archivo `Evaluación del Pitch (2021.04.13 ISoftware).csv` contiene las columnas `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto, `"Equipo que vas a evaluar:"` equipo que ha expuesto y que va a ser evaluado por cada estudiante que no sea integrante; a continuación, se tienen las siguientes columnas que corresponden a la evaluación de los respectivos ítems:

    1. "Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?",
    2. "Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?",
    3. "Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?",
    4. "Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?",
    5. "Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?",
    6. "Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.",
    7. "Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.",
    8. "Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.",
    9. "Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?",
    10. "Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ",
    11. "Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?",
    12. "Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.",
    13. "Exposición: ¿Qué tan coordinados estaban los expositores?",
    14. "Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?",
    15. "Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.",
    * "Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)",
    * "Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación."


* Cada ítem se evalúa con la siguiente escala: 0. Ausente; 1. Deficiente; 2. Regular; 3. Aceptable; 4. Bueno; 5. Excelente

* Carge los datos, cada archivo en una tabla SparkSQL y responda cada una de las consultas dadas en cada celda. Tenga en cuenta que algunas consultas pueden tener como resultado el vacío.

# Integrantes del equipo
1. Andrea Stephanya Pino Villegas - C.C 1.144.158.067
2. Edward Darío Mercado Gómez - C.C. 1.038.131.004
3. Daniel Escobar Ruiz - C.C. 1.040.747.009
4. Daniela Rivera Vargas - C.C. 1.037.636.542
5. Simon Pineda Agudelo - C.C. 1.152.220.029

In [1]:
# Agrege acá el código para importar las librerias
!pip3 install Cython
!pip3 install pyarrow==0.9.0
!pip3 install ts-flint
!wget https://repo1.maven.org/maven2/com/twosigma/flint/0.6.0/flint-0.6.0.jar

    100% |################################| 2.0MB 591kB/s ta 0:00:01
    100% |################################| 13.7MB 98kB/s eta 0:00:01
--2021-05-23 19:31:54--  https://repo1.maven.org/maven2/com/twosigma/flint/0.6.0/flint-0.6.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 199.232.192.209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2171677 (2.1M) [application/java-archive]
Saving to: 'flint-0.6.0.jar'

flint-0.6.0.jar     100%[===================>]   2.07M  1.30MB/s    in 1.6s    

2021-05-23 19:31:56 (1.30 MB/s) - 'flint-0.6.0.jar' saved [2171677/2171677]



In [2]:
import os
# Requerido https://github.com/twosigma/flint/blob/master/python/README.md
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /datalake/flint-0.6.0.jar --py-files /datalake/flint-0.6.0.jar pyspark-shell'

In [3]:
# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql import functions as F

In [4]:
# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

In [5]:
# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

In [6]:
# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

In [7]:
# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [8]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...
!hdfs dfs -put asistencia.csv
!hdfs dfs -put evaluacion.csv
!hdfs dfs -ls

Found 2 items
-rw-r--r--   1 root supergroup      93077 2021-05-23 19:32 asistencia.csv
-rw-r--r--   1 root supergroup     110401 2021-05-23 19:32 evaluacion.csv


In [9]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...

ContextoSql.sql('DROP TABLE IF EXISTS asistencia')
ContextoSql.sql('DROP TABLE IF EXISTS evaluacion')

ContextoSql.sql("""
CREATE TABLE
asistencia (
        timestamp TIMESTAMP,
        email STRING,
        equipo_pertenezco STRING,
        equipo_expone STRING
    ) 
USING CSV OPTIONS (
        header='true',
        multiLine='true',
        path='asistencia.csv',
        nullvalue='NA',
        timestampFormat="yyyy/MM/dd HH:mm:ss"
    )
""");

ContextoSql.sql("""
CREATE TABLE 
evaluacion (
        timestamp TIMESTAMP,
        email STRING,
        equipo_evaluado STRING,
        introduccion FLOAT,
        equipo FLOAT,
        problema FLOAT,
        ventajas FLOAT,
        solucion FLOAT,
        producto FLOAT,
        traccion FLOAT,
        mercado FLOAT,
        competencia FLOAT,
        modelo FLOAT,
        inversion FLOAT,
        contacto FLOAT,
        expo_coordinacion FLOAT,
        expo_claridad FLOAT,
        expo_material FLOAT,
        invierte STRING,
        observaciones STRING
    )
USING CSV OPTIONS (
        multiLine='true',
        path='evaluacion.csv',
        nullvalue='NA', 
        timestampFormat="yyyy/MM/dd HH:mm:ss", 
        header 'true'
    )
""");

In [82]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...
consulta_estudiantes = ContextoSql.sql("SELECT email FROM evaluacion GROUP BY email ORDER BY email ASC")
consulta_estudiantes.show(5)

+--------------------+
|               email|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
+--------------------+
only showing top 5 rows



In [83]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

tabla_asistencia = ContextoSql.table('asistencia')
asistencia = tabla_asistencia.groupBy('email').agg(F.min('timestamp').alias('primera_asistencia'), F.max('timestamp').alias('ultima_asistencia'))
asistencia.show(5)

+--------------------+-------------------+-------------------+
|               email| primera_asistencia|  ultima_asistencia|
+--------------------+-------------------+-------------------+
|samolinap@unal.ed...|2021-04-08 08:11:43|2021-04-13 09:37:26|
| jruedat@unal.edu.co|2021-04-08 08:12:55|2021-04-13 09:41:27|
|nvalenciat@unal.e...|2021-04-08 08:10:18|2021-04-13 09:39:34|
|serendona@unal.ed...|2021-04-08 08:10:47|2021-04-13 09:40:25|
|jgutierrezlo@unal...|2021-04-08 08:12:09|2021-04-13 09:39:06|
+--------------------+-------------------+-------------------+
only showing top 5 rows



In [86]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...

consulta_asistencia_menos = ContextoSql.sql("SELECT email, COUNT(email) FROM asistencia GROUP BY email HAVING COUNT(email) <= 2")
consulta_asistencia_menos.show()

+--------------------+------------+
|               email|count(email)|
+--------------------+------------+
|jdroldano@unal.ed...|           1|
+--------------------+------------+



In [87]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...

consulta_sin_asistencia = ContextoSql.sql("SELECT email, COUNT(email) FROM asistencia GROUP BY email HAVING COUNT(email) = 0")
consulta_sin_asistencia.show()

+-----+------------+
|email|count(email)|
+-----+------------+
+-----+------------+



In [88]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...

consulta_integrantes = ContextoSql.sql("SELECT equipo_pertenezco, email FROM asistencia GROUP BY equipo_pertenezco, email ORDER BY equipo_pertenezco")
consulta_integrantes.show(5)

+--------------------+--------------------+
|   equipo_pertenezco|               email|
+--------------------+--------------------+
|Pertenezco a: Adm...| jortize@unal.edu.co|
|Pertenezco a: Adm...|nsalinas@unal.edu.co|
|Pertenezco a: Adm...|saespinosab@unal....|
|Pertenezco a: Adm...|rmbuilesm@unal.ed...|
|Pertenezco a: Adm...|jhcordoba@unal.ed...|
+--------------------+--------------------+
only showing top 5 rows



In [89]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...

# consulta_asistentes_no_equipo = ContextoSql.sql("SELECT equipo_evaluado, COUNT(email), email FROM asistencia GROUP BY equipo_pertenezco, email ORDER BY equipo_pertenezco")
tabla_asistencia = ContextoSql.table('asistencia')
consulta = tabla_asistencia.select(['equipo_pertenezco', 'email']).filter(tabla_asistencia['equipo_pertenezco'].substr(14, 100) != tabla_asistencia['equipo_expone'].substr(8, 100))
consulta.show(5)

+--------------------+--------------------+
|   equipo_pertenezco|               email|
+--------------------+--------------------+
|Pertenezco a: Ges...|auarbelaeza@unal....|
|Pertenezco a: Lo ...|jmunozhe@unal.edu.co|
|Pertenezco a: Mi ...|fmiranda@unal.edu.co|
|Pertenezco a: Ges...|jucardonaa@unal.e...|
|Pertenezco a: Ges...|seagudeloo@unal.e...|
+--------------------+--------------------+
only showing top 5 rows



In [90]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...

tabla_asistencia = ContextoSql.table('asistencia')
consulta = tabla_asistencia.select(['equipo_pertenezco', 'email']).filter(tabla_asistencia['equipo_pertenezco'].substr(14, 100) == tabla_asistencia['equipo_expone'].substr(8, 100)).show(5)

+--------------------+--------------------+
|   equipo_pertenezco|               email|
+--------------------+--------------------+
|Pertenezco a: Ser...|jppinedal@unal.ed...|
|Pertenezco a: Lo ...|dgarciabl@unal.ed...|
|Pertenezco a: Te ...|dgerenal@unal.edu.co|
|Pertenezco a: Te ...|nvalenciat@unal.e...|
|Pertenezco a: Te ...|juortizt@unal.edu.co|
+--------------------+--------------------+
only showing top 5 rows



In [92]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...
email_equipos = ContextoSql.table('asistencia').select('email', 'equipo_pertenezco').distinct()

items = [
    'introduccion',
    'equipo',
    'problema',
    'ventajas',
    'solucion',
    'producto',
    'traccion',
    'mercado',
    'competencia',
    'modelo',
    'inversion',
    'contacto',
    'expo_coordinacion',
    'expo_claridad',
    'expo_material']

tabla_evaluacion = ContextoSql.table('evaluacion').select(['email', 'equipo_evaluado'] + items).join(email_equipos, on=['email'], how='left')

tabla_evaluacion_limpia = tabla_evaluacion.filter(tabla_evaluacion['equipo_evaluado'] != tabla_evaluacion['equipo_pertenezco'].substr(14, 100))

promedio_items = tabla_evaluacion.groupBy('equipo_evaluado').agg({ item: 'avg' for item in items })

notas_promedio = promedio_items.withColumn('nota_promedio_total', sum(promedio_items['avg(' + str(col) + ')'] for col in items) / 15)

notas_promedio.show(truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------
 equipo_evaluado        | Servicios a domicilio                                  
 avg(traccion)          | 4.144736842105263                                      
 avg(expo_coordinacion) | 4.4868421052631575                                     
 avg(ventajas)          | 4.2631578947368425                                     
 avg(solucion)          | 4.5131578947368425                                     
 avg(competencia)       | 4.328947368421052                                      
 avg(equipo)            | 4.618421052631579                                      
 avg(inversion)         | 4.157894736842105                                      
 avg(modelo)            | 4.2894736842105265                                     
 avg(expo_material)     | 4.552631578947368                                      
 avg(introduccion)      | 4.4605263157894735                                     
 avg(mercado)   

In [93]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...
items_avg = [
    'avg(introduccion)',
    'avg(equipo)',
    'avg(problema)',
    'avg(ventajas)',
    'avg(solucion)',
    'avg(producto)',
    'avg(traccion)',
    'avg(mercado)',
    'avg(competencia)',
    'avg(modelo)',
    'avg(inversion)',
    'avg(contacto)',
    'avg(expo_coordinacion)',
    'avg(expo_claridad)',
    'avg(expo_material)',
    'nota_promedio_total']

for item in items_avg:
    notas_promedio.select(['equipo_evaluado', item]).filter(notas_promedio[item] == notas_promedio.select(item).rdd.max()[0]).show()

+--------------------+-----------------+
|     equipo_evaluado|avg(introduccion)|
+--------------------+-----------------+
|Gesti��n de Resid...|4.561643835616438|
+--------------------+-----------------+

+--------------------+-----------+
|     equipo_evaluado|avg(equipo)|
+--------------------+-----------+
|Deserci��n de est...|        5.0|
+--------------------+-----------+

+--------------------+-------------+
|     equipo_evaluado|avg(problema)|
+--------------------+-------------+
|Deserci��n de est...|          5.0|
+--------------------+-------------+

+--------------------+-------------+
|     equipo_evaluado|avg(ventajas)|
+--------------------+-------------+
|Deserci��n de est...|          5.0|
+--------------------+-------------+

+--------------------+------------------+
|     equipo_evaluado|     avg(solucion)|
+--------------------+------------------+
|Servicios a domic...|4.5131578947368425|
+--------------------+------------------+

+--------------------+-------------

In [94]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...

for item in items_avg:
    notas_promedio.select(['equipo_evaluado', item]).filter(notas_promedio[item] == notas_promedio.select(item).rdd.min()[0]).show()

+--------------------+-----------------+
|     equipo_evaluado|avg(introduccion)|
+--------------------+-----------------+
|Administraci��n d...|4.135135135135135|
+--------------------+-----------------+

+---------------+-----------------+
|equipo_evaluado|      avg(equipo)|
+---------------+-----------------+
|Mi Sanandresito|3.935064935064935|
+---------------+-----------------+

+--------------------+------------------+
|     equipo_evaluado|     avg(problema)|
+--------------------+------------------+
|Administraci��n d...|3.9864864864864864|
+--------------------+------------------+

+--------------------+------------------+
|     equipo_evaluado|     avg(ventajas)|
+--------------------+------------------+
|Gesti��n de progr...|3.9342105263157894|
+--------------------+------------------+

+--------------------+-----------------+
|     equipo_evaluado|    avg(solucion)|
+--------------------+-----------------+
|Administraci��n d...|4.162162162162162|
+--------------------+-----

In [81]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

#Revisar

notas = notas_promedio.select('equipo_evaluado', 'nota_promedio_total')
notas = notas.withColumnRenamed('equipo_evaluado', 'equipo_pertenezco')

equipos = ContextoSql.table('asistencia').select(['email', 'equipo_pertenezco']).distinct()
equipos_limpios = equipos.withColumn('equipo_pertenezco', equipos['equipo_pertenezco'].substr(15, 100))

notas_estudiantes = equipos_limpios.join(notas, on=['equipo_pertenezco'], how='left').select(['email', 'nota_promedio_total'])
notas_estudiantes.show(100)

+--------------------+-------------------+
|               email|nota_promedio_total|
+--------------------+-------------------+
|anapariciom@unal....|  4.419298245614034|
|sacastrot@unal.ed...|  4.419298245614034|
|sebermudezg@unal....|  4.419298245614034|
|cjfunezg@unal.edu.co|  4.419298245614034|
|dosoriom@unal.edu.co|  4.419298245614034|
| bocampo@unal.edu.co|  4.419298245614034|
|jmunozbo@unal.edu.co|  4.419298245614034|
| jruedat@unal.edu.co|  4.419298245614034|
|jppinedal@unal.ed...|  4.419298245614034|
|jdroldano@unal.ed...|  4.419298245614034|
|emflorezb@unal.ed...|    4.2009009009009|
|nsalinas@unal.edu.co|    4.2009009009009|
| jortize@unal.edu.co|    4.2009009009009|
|rmbuilesm@unal.ed...|    4.2009009009009|
|samolinap@unal.ed...|    4.2009009009009|
| kmolano@unal.edu.co|    4.2009009009009|
|sednarvaezna@unal...|    4.2009009009009|
|angutierrezb@unal...|    4.2009009009009|
|lumarinb@unal.edu.co|    4.2009009009009|
|jhcordoba@unal.ed...|    4.2009009009009|
|saespinosa